In [16]:
import pandas as pd
import numpy as np
import sys

!{sys.executable} -m pip install -q scikit-learn

# CSV yolunu ayarla
csv_path = "Police_Transparency_-_Arrests_-_All_Data_(main_table___denormalized).csv"

try:
    df = pd.read_csv(csv_path, low_memory=False)
    print("✔ Veri seti başarıyla yüklendi.")
except Exception as e:
    print(f"❌ Veri yükleme hatası: {e}")
    raise SystemExit

print(f"Veri boyutu: {df.shape}")
display(df.head())


✔ Veri seti başarıyla yüklendi.
Veri boyutu: (47444, 40)


,X,Y,rin,primary_key,arrest_type,arrest_translation,arrest_dt,arrest_time,arrest_hour_of_day,location,...,severity_code,severity_trans,ChargeClassTranslation,ChargeGrouping,arrestee_sex,arrestee_race,arrestee_ethnicity,arrestee_genderTranslation,arrestee_RaceAndEthnicity,ESRI_OID
0,693985,882475,110528,TE20226631,O,On-View: Arrested when first observed/investi...,2022/09/24 21:17:00+00,2117,21,2XX E 5TH ST,...,NaN,Unknown,Arizona Revised Statutes,Miscellaneous,M,B,N,Male,Black or African American,3715748
1,710663,880237,110739,TE20226842,T,Taken Into Custody: Arrest on warrant or PC f...,2022/10/04 17:30:00+00,1730,17,9XX S ACORN AVE,...,F,Felony,Arizona Revised Statutes,Drug charges,M,W,N,Male,White,3715749
2,704259,878441,110587,TE20226690,T,Taken Into Custody: Arrest on warrant or PC f...,2022/09/27 21:52:00+00,2152,21,1XXX E APACHE BLVD,...,M,Misdemeanor,Arizona Revised Statutes,Assault & related charges,M,W,H,Male,Hispanic or Latino,3715750
3,693160,868321,110521,TE20226624,O,On-View: Arrested when first observed/investi...,2022/09/24 21:00:00+00,2100,21,4XXX S MILL AVE,...,F,Felony,Arizona Revised Statutes,Miscellaneous,M,B,N,Male,Black or African American,3715751
4,708569,883842,110789,TE20226892,T,Taken Into Custody: Arrest on warrant or PC f...,2022/10/07 03:54:00+00,354,3,2XXX W RIO SALADO PKWY,...,M,Misdemeanor,Arizona Revised Statutes,Escape & related charges,M,W,H,Male,Hispanic or Latino,3715752


In [2]:
print("\n--- TEKİLLEŞTİRME ---")

initial_rows = df.shape[0]
df.drop_duplicates(inplace=True)
removed = initial_rows - df.shape[0]

print(f"✔ {removed} adet yinelenen satır silindi.")
print(f"Yeni veri boyutu: {df.shape}")



--- TEKİLLEŞTİRME ---
✔ 0 adet yinelenen satır silindi.
Yeni veri boyutu: (47444, 40)


In [3]:
columns_to_drop = [
    'X','Y','rin','primary_key','charge_rin','pin','ESRI_OID',
    'x_coordinate','y_coordinate',
    'arrest_officer',
    'ofc_age_range','ofc_genderTranslation','ofc_RaceAndEthnicity',
    'statute','class','severity_code'
]

existing_cols = [col for col in columns_to_drop if col in df.columns]
df.drop(columns=existing_cols, inplace=True)

print(f"✔ {len(existing_cols)} sütun kaldırıldı.")
print("Kaldırılanlar:", existing_cols)


✔ 16 sütun kaldırıldı.
Kaldırılanlar: ['X', 'Y', 'rin', 'primary_key', 'charge_rin', 'pin', 'ESRI_OID', 'x_coordinate', 'y_coordinate', 'arrest_officer', 'ofc_age_range', 'ofc_genderTranslation', 'ofc_RaceAndEthnicity', 'statute', 'class', 'severity_code']


In [4]:
missing = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)
display(missing.head(20))
print("\nVeri boyutu:", df.shape)


area_name                     5.667735
zipcode                       5.225107
arrest_translation            1.199309
arrest_type                   1.199309
ChargeGrouping                0.265576
arrestee_ethnicity            0.073771
arrestee_age_range            0.067448
arrestee_RaceAndEthnicity     0.040047
arrestee_genderTranslation    0.040047
arrestee_race                 0.040047
arrestee_sex                  0.040047
charge_count                  0.002108
ChargeClassTranslation        0.000000
severity_trans                0.000000
arrest_hour_of_day            0.000000
location                      0.000000
arrest_dt                     0.000000
arrest_time                   0.000000
grid                          0.000000
zone                          0.000000
dtype: float64


Veri boyutu: (47444, 24)


In [5]:
print("\n--- Aykırı Değer İşleme ---")

if "charge_count" in df.columns:
    Q1 = df['charge_count'].quantile(0.25)
    Q3 = df['charge_count'].quantile(0.75)
    IQR = Q3 - Q1
    upper = Q3 + 1.5*IQR

    anomalies = df[df['charge_count'] > upper].shape[0]

    df.loc[df['charge_count'] > upper, 'charge_count'] = np.nan
    print(f"✔ {anomalies} adet aykırı değer NaN yapıldı.")
else:
    print("⚠ 'charge_count' bulunamadı.")



--- Aykırı Değer İşleme ---
✔ 856 adet aykırı değer NaN yapıldı.


In [6]:
categorical_cols = df.select_dtypes(include=['object','category','bool']).columns

threshold = 0.001
print(f"\n--- Düşük Frekanslı Kategoriler (%{threshold*100}) ---")

for col in categorical_cols:
    vc = df[col].value_counts(normalize=True)
    low = vc[vc < threshold].index.tolist()
    if low:
        print(f"• {col}: {len(low)} düşük frekanslı kategori → {low[:3]}...")



--- Düşük Frekanslı Kategoriler (%0.1) ---
• arrest_type: 1 düşük frekanslı kategori → ['S']...
• arrest_translation: 1 düşük frekanslı kategori → ['Summoned/Cited: Ticket issued, or long form charges approved, not taken to jail']...
• arrest_dt: 20549 düşük frekanslı kategori → ['2023/02/11 20:00:00+00', '2024/05/09 16:10:00+00', '2023/06/02 10:14:00+00']...
• location: 2015 düşük frekanslı kategori → ['4XX W BROADWAY RD    ', '1XXX E VISTA DEL CERRO DR     ', ' PRIEST DR / W SOUTHERN AVE  ']...
• municipality: 2 düşük frekanslı kategori → ['SR        ', 'CH        ']...
• district: 2 düşük frekanslı kategori → ['UI    ', 'DT    ']...
• zone: 2 düşük frekanslı kategori → ['UI    ', '1     ']...
• grid: 223 düşük frekanslı kategori → ['2311  ', '0606  ', '1317  ']...
• charge: 5889 düşük frekanslı kategori → ['PRESCRIPT DRUG-POSSESS/USE                                                                                                                                                       

In [7]:
print("\n--- İmputasyon Başladı (Uyarısız Güvenli Versiyon) ---")

# SAYISAL İMPUTASYON (MOD)
num_cols = df.select_dtypes(include=np.number).columns

for col in num_cols:
    if df[col].isna().any():
        mode_val = df[col].mode(dropna=True)[0]
        # inplace kullanılmıyor = SettingWithCopyWarning yok
        df[col] = df[col].fillna(mode_val)
        print(f"✔ '{col}' mod ({mode_val}) ile dolduruldu.")

# KATEGORİK İMPUTASYON (DAĞILIM KORUMA)
cat_cols = df.select_dtypes(include=['object','category','bool']).columns

def impute_with_dist(col):
    nonnull = df[col].dropna()
    if nonnull.empty:
        return  # tüm kolon NaN ise
    probs = nonnull.value_counts(normalize=True)
    nan_mask = df[col].isna()
    nan_count = nan_mask.sum()
    if nan_count == 0:
        return
    random_values = np.random.choice(probs.index, size=nan_count, p=probs.values)
    # Kritik kısım: df.loc[...] = ... → SettingWithCopyWarning yok çünkü df üzerinden işlem
    df.loc[nan_mask, col] = random_values
    print(f"✔ '{col}' kategorik imputasyon ile dolduruldu.")

for col in cat_cols:
    if df[col].isna().any():
        impute_with_dist(col)

print("\n✔ Tüm imputasyonlar uyarısız olarak tamamlandı.")



--- İmputasyon Başladı (Uyarısız Güvenli Versiyon) ---
✔ 'zipcode' mod (85281.0) ile dolduruldu.
✔ 'charge_count' mod (1.0) ile dolduruldu.
✔ 'arrest_type' kategorik imputasyon ile dolduruldu.
✔ 'arrest_translation' kategorik imputasyon ile dolduruldu.
✔ 'area_name' kategorik imputasyon ile dolduruldu.
✔ 'arrestee_age_range' kategorik imputasyon ile dolduruldu.
✔ 'ChargeGrouping' kategorik imputasyon ile dolduruldu.
✔ 'arrestee_sex' kategorik imputasyon ile dolduruldu.
✔ 'arrestee_race' kategorik imputasyon ile dolduruldu.
✔ 'arrestee_ethnicity' kategorik imputasyon ile dolduruldu.
✔ 'arrestee_genderTranslation' kategorik imputasyon ile dolduruldu.
✔ 'arrestee_RaceAndEthnicity' kategorik imputasyon ile dolduruldu.

✔ Tüm imputasyonlar uyarısız olarak tamamlandı.


In [8]:
print("\n--- Zaman Feature Engineering ---")

if 'arrest_dt' in df.columns:
    df['arrest_dt'] = pd.to_datetime(df['arrest_dt'].astype(str).str.split().str[0], errors='coerce')
    df['arrest_year'] = df['arrest_dt'].dt.year
    df['arrest_month'] = df['arrest_dt'].dt.month
    df['arrest_day_of_week'] = df['arrest_dt'].dt.dayofweek

    def get_time_cat(h):
        if pd.isna(h): return 'Bilinmiyor'
        if 0<=h<=6: return 'Gece'
        if 7<=h<=11: return 'Sabah'
        if 12<=h<=16: return 'Ogle'
        if 17<=h<=20: return 'Ikindi'
        return 'Aksam'

    df['time_category'] = df['arrest_hour_of_day'].apply(get_time_cat)

    df.drop(columns=[c for c in ['arrest_dt','arrest_time'] if c in df.columns], inplace=True)
    print("✔ Zaman özellikleri eklendi.")



--- Zaman Feature Engineering ---
✔ Zaman özellikleri eklendi.


In [9]:
print("\n--- Hedef Değişken ---")

df['is_onview_arrest'] = np.where(df['arrest_type']=='O', 1, 0)
df.drop(columns=[c for c in ['arrest_type','arrest_translation'] if c in df.columns],
        inplace=True)

print("✔ 'is_onview_arrest' oluşturuldu.")



--- Hedef Değişken ---
✔ 'is_onview_arrest' oluşturuldu.


In [10]:
ohe_cols = [c for c in df.select_dtypes(include=['object','category','bool'])
            if df[c].nunique() < 50]

df = pd.get_dummies(df, columns=ohe_cols, drop_first=True, dtype=int)

print(f"✔ OHE tamamlandı. Yeni sütun sayısı: {df.shape[1]}")


✔ OHE tamamlandı. Yeni sütun sayısı: 112


In [11]:
from sklearn.preprocessing import StandardScaler

scale_cols = [c for c in ['charge_count','arrest_hour_of_day'] if c in df.columns]

scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])

print("✔ StandardScaler uygulandı.")


✔ StandardScaler uygulandı.


In [21]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer  # <--- YENİ EKLENDİ: Boşluk Doldurucu
import pandas as pd
import numpy as np
import sys

print(">>> MELİH İÇİN ANALİZ BAŞLIYOR...")

# -----------------------------------------------------------------------------
# ADIM 0: VERİ HAZIRLIĞI VE HEDEF DEĞİŞKEN KONTROLÜ
# -----------------------------------------------------------------------------

if 'df' not in locals():
    print("❌ HATA: Veri seti (df) hafızada yok! Lütfen dosyanın en başındaki veri yükleme kodlarını çalıştır.")
else:
    # Hedef değişken yoksa oluştur
    if 'is_onview_arrest' not in df.columns:
        print("⚠ UYARI: 'is_onview_arrest' sütunu eksik. Oluşturuluyor...")
        if 'arrest_type' in df.columns:
            df['is_onview_arrest'] = np.where(df['arrest_type'] == 'O', 1, 0)
            print("   ✔ Hedef değişken oluşturuldu.")
        else:
            print("❌ KRİTİK HATA: 'arrest_type' sütunu da yok. Veri seti bozuk.")

# -----------------------------------------------------------------------------
# BÖLÜM 1: EKSİK VERİ DOLDURMA & FEATURE SELECTION
# -----------------------------------------------------------------------------
print("\n--- 🛠 VERİ TEMİZLİĞİ VE VARIANCE THRESHOLD ---")

# 1. X ve Y'yi Ayır
try:
    X = df.drop(columns=['is_onview_arrest'])
    Y = df['is_onview_arrest']
except KeyError:
    print("❌ HATA: 'is_onview_arrest' sütunu bulunamadı.")
    raise

# 2. Sadece Sayısal Verileri Al (Modeller matematik sever)
X_numeric_raw = X.select_dtypes(include=np.number)

# 3. EKSİK VERİ DOLDURMA (IMPUTATION) - KRİTİK ADIM
# Gradient Boosting boş veri sevmez. Boşlukları ortalama ile dolduruyoruz.
print(f"   Bilgi: İşlenen sütun sayısı: {X_numeric_raw.shape[1]}")
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X_numeric_raw), columns=X_numeric_raw.columns)

# 4. ÖLÇEKLENDİRME (SCALING)
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X_numeric_raw.columns)

# 5. FEATURE SELECTION (Variance Threshold)
print("   📉 Variance Threshold Uygulanıyor (Eşik: 0.01)...")
selector = VarianceThreshold(threshold=0.01)
selector.fit(X_scaled)


X_fs_array = selector.transform(X_scaled)
secilen_ozellikler = X_scaled.columns[selector.get_support(indices=True)]
X_fs = pd.DataFrame(X_fs_array, columns=secilen_ozellikler)

print(f"✔ Başlangıç Özellik Sayısı: {X_scaled.shape[1]}")
print(f"✔ Seçilen Özellik Sayısı: {X_fs.shape[1]}")
print(f"✔ Elenen (Gereksiz) Özellik Sayısı: {X_scaled.shape[1] - X_fs.shape[1]}")


# -----------------------------------------------------------------------------
# BÖLÜM 2: VERİ SETİNİ BÖLME
# -----------------------------------------------------------------------------
# Stratify=Y diyerek veriyi dengeli bölüyoruz (Suçlu oranı her parçada eşit olsun)

# 1. FS Öncesi (Tüm özellikler)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.25, random_state=42, stratify=Y)

# 2. FS Sonrası (Seçilenler)
X_train_fs, X_test_fs, y_train_fs, y_test_fs = train_test_split(X_fs, Y, test_size=0.25, random_state=42, stratify=Y)


# -----------------------------------------------------------------------------
# BÖLÜM 3: MODEL EĞİTİMİ (DECISION TREE & GRADIENT BOOSTING)
# -----------------------------------------------------------------------------

def evaluate_model(model, X_tr, X_te, y_tr, y_te):
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)
    try:
        y_prob = model.predict_proba(X_te)[:, 1]
        roc = roc_auc_score(y_te, y_prob)
    except:
        roc = 0 
    
    acc = accuracy_score(y_te, y_pred)
    prec = precision_score(y_te, y_pred, zero_division=0)
    rec = recall_score(y_te, y_pred, zero_division=0)
    f1 = f1_score(y_te, y_pred, zero_division=0)
    
    return [acc, prec, rec, f1, roc]

results = []
print("\n>>> Modeller Eğitiliyor...")

# --- MODEL 1: DECISION TREE ---
dt = DecisionTreeClassifier(random_state=42, class_weight='balanced')
results.append(["Decision Tree", "FS Öncesi", *evaluate_model(dt, X_train, X_test, y_train, y_test)])
results.append(["Decision Tree", "FS Sonrası", *evaluate_model(dt, X_train_fs, X_test_fs, y_train_fs, y_test_fs)])

# --- MODEL 2: GRADIENT BOOSTING ---
gb = GradientBoostingClassifier(random_state=42)
results.append(["Gradient Boosting", "FS Öncesi", *evaluate_model(gb, X_train, X_test, y_train, y_test)])
results.append(["Gradient Boosting", "FS Sonrası", *evaluate_model(gb, X_train_fs, X_test_fs, y_train_fs, y_test_fs)])


# -----------------------------------------------------------------------------
# BÖLÜM 4: SONUÇ TABLOSU
# -----------------------------------------------------------------------------
comparison_df = pd.DataFrame(
    results,
    columns=["Model", "Aşama", "Accuracy", "Precision", "Recall", "F1 Score", "ROC-AUC"]
)

print("\n\n################################################################")
print("📢 MELİH - MODEL SONUÇLARI (DECISION TREE & GRADIENT BOOSTING)")
print("################################################################")
display(comparison_df)

>>> MELİH İÇİN ANALİZ BAŞLIYOR...

--- 🛠 VERİ TEMİZLİĞİ VE VARIANCE THRESHOLD ---
   Bilgi: İşlenen sütun sayısı: 11
   📉 Variance Threshold Uygulanıyor (Eşik: 0.01)...
✔ Başlangıç Özellik Sayısı: 11
✔ Seçilen Özellik Sayısı: 8
✔ Elenen (Gereksiz) Özellik Sayısı: 3

>>> Modeller Eğitiliyor...


################################################################
📢 MELİH - MODEL SONUÇLARI (DECISION TREE & GRADIENT BOOSTING)
################################################################


,Model,Aşama,Accuracy,Precision,Recall,F1 Score,ROC-AUC
0,Decision Tree,FS Öncesi,0.908861,0.808413,0.809271,0.808842,0.874641
1,Decision Tree,FS Sonrası,0.896973,0.779833,0.790870,0.785313,0.860515
2,Gradient Boosting,FS Öncesi,0.774893,0.736364,0.085987,0.153992,0.729736
3,Gradient Boosting,FS Sonrası,0.769750,0.693878,0.060156,0.110713,0.708365
